## DATA EXPLORATION

In [1]:
#imports
import pandas as pd

In [3]:
alerts_cleaned_df = pd.read_csv('../data/raw/mapmf_alerts_cleaned.csv')
dict_df = pd.read_csv('../data/raw/mapmf_data_dictionary.csv')

In [4]:
alerts_cleaned_df.head()

,id,title,date,year,month,day_of_week,country,region_level_1,region_level_2,primary_incident_type,...,type_of_incident,primary_source,source_of_incident,context_of_incident,region_names,content,content_length,published_at_date,_geo_lat,_geo_lng
0,34388,Journalist Emilia Șercan targeted with smear c...,2026-01-12,2026,1,Monday,Romania,Romania,Bucharest,Intimidation/threatening,...,Intimidation/threatening | Verbal attack | Dis...,Media outlet,Media outlet | Legislation: government/public ...,Online/digital,EU Member States | Romania | Bucharest,"On 12 January 2026, investigative journalist E...",1740,2026-01-15,44.436141,26.102684
1,34382,OCCRP website targeted with DDoS attack,2026-01-12,2026,1,Monday,Netherlands,Netherlands,North Holland,Hacking/DDoS,...,Hacking/DDoS | Attack to property | Blocked di...,Unknown source of incident,Unknown source of incident,Online/digital,EU Member States | Netherlands | North Holland,"On 13 January 2026, the Organised Crime and Co...",1019,2026-01-14,52.373080,4.892453
2,34358,Slovak political commentator Peter Schutz viol...,2026-01-10,2026,1,Saturday,Slovakia,Slovakia,Eastern Slovakia,Injury (physical assault resulting in injury),...,Injury (physical assault resulting in injury) ...,Unknown source of incident,Unknown source of incident,Public place/street,EU Member States | Slovakia | Eastern Slovakia...,"On 10 January 2025, Slovak political commentat...",2401,2026-01-12,48.717227,21.249677
3,34392,Ici Nord Sylvain Charley cyber harassed,2026-01-08,2026,1,Thursday,France,France,Hauts-de-France,"Harassment, insult, bullying",...,"Harassment, insult, bullying | Verbal attack",Private individual(s),Private individual(s),Online/digital,EU Member States | France | Hauts-de-France | ...,"On 8 January 2026, Sylvain Charley, a journali...",1045,2026-01-16,50.636565,3.063528
4,34356,Police opens investigation into alleged survei...,2026-01-08,2026,1,Thursday,Ukraine,Ukraine,Київська міська громада,Surveillance and interception of journalistic ...,...,Surveillance and interception of journalistic ...,Private individual(s),Private individual(s),Online/digital,EU candidate countries | Ukraine | Київська мі...,"On 8 January 2026, Ukrainian authorities repor...",1865,2026-01-12,50.450034,30.524136


In [5]:
alerts_cleaned_df.shape

(11305, 21)

In [6]:
alerts_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11305 entries, 0 to 11304
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     11305 non-null  int64  
 1   title                  11305 non-null  object 
 2   date                   11305 non-null  object 
 3   year                   11305 non-null  int64  
 4   month                  11305 non-null  int64  
 5   day_of_week            11305 non-null  object 
 6   country                11305 non-null  object 
 7   region_level_1         11305 non-null  object 
 8   region_level_2         11174 non-null  object 
 9   primary_incident_type  11305 non-null  object 
 10  incident_type_count    11305 non-null  int64  
 11  type_of_incident       11305 non-null  object 
 12  primary_source         11305 non-null  object 
 13  source_of_incident     11305 non-null  object 
 14  context_of_incident    11305 non-null  object 
 15  re

In [7]:
dict_df.head()

,column_name,data_type,description,notes
0,id,integer,Unique identifier for each alert,Primary key from MAPMF database
1,title,string,Brief headline describing the incident,May contain journalist names and incident type
2,date,date (YYYY-MM-DD),Date when the incident occurred,Incident date (not publication date)
3,year,integer,Year extracted from the incident date,Range: 2014-2026
4,month,integer,Month extracted from the incident date (1-12),1=January through 12=December


In [8]:
dict_df.shape

(21, 4)

In [9]:
alerts_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11305 entries, 0 to 11304
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     11305 non-null  int64  
 1   title                  11305 non-null  object 
 2   date                   11305 non-null  object 
 3   year                   11305 non-null  int64  
 4   month                  11305 non-null  int64  
 5   day_of_week            11305 non-null  object 
 6   country                11305 non-null  object 
 7   region_level_1         11305 non-null  object 
 8   region_level_2         11174 non-null  object 
 9   primary_incident_type  11305 non-null  object 
 10  incident_type_count    11305 non-null  int64  
 11  type_of_incident       11305 non-null  object 
 12  primary_source         11305 non-null  object 
 13  source_of_incident     11305 non-null  object 
 14  context_of_incident    11305 non-null  object 
 15  re